# Portfolio Optimization using MPT 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# --- Parameters ---
trading_days = 252
n_portfolios = 20000   # Monte Carlo sample size; increase for smoother frontier
risk_free_rate = 0.02  # annual risk-free rate used for Sharpe ratio (2%)

# --- 0. Load return series for assets from your Task1 assets dict ---
tsla = assets['TSLA']['Adj Close'].copy()
bnd  = assets['BND']['Adj Close'].copy()
spy  = assets['SPY']['Adj Close'].copy()

# Align by date (inner join)
prices = pd.concat([tsla, bnd, spy], axis=1, keys=['TSLA','BND','SPY'])
prices.columns = prices.columns.droplevel(0)  # keep simple column names
prices = prices.dropna()  # align to common trading days

# Compute daily returns
daily_returns = prices.pct_change().dropna()

# --- 1. Expected returns vector ---
# TSLA expected return from forecast if present
def tsla_expected_return_from_forecast():
    # try ARIMA then LSTM then combined
    # expected return = annualized mean daily return of forecasted mean series (percentage change)
    try:
        # if combined DataFrame exists (Task3)
        # combined should have 'LSTM_mean' or 'ARIMA_mean' columns
        if 'combined' in globals():
            # prefer the best-performing model — if unknown, pick ARIMA_mean if present
            col = None
            if 'ARIMA_mean' in combined.columns:
                col = 'ARIMA_mean'
            elif 'LSTM_mean' in combined.columns:
                col = 'LSTM_mean'
            if col:
                fc = combined[col]
                # compute daily returns of forecast (pct change)
                fc_ret = fc.pct_change().dropna()
                return fc_ret.mean() * trading_days  # annualize
        # try arima_forecast_df
        if 'arima_forecast_df' in globals():
            fc = arima_forecast_df['mean']
            fc_ret = fc.pct_change().dropna()
            return fc_ret.mean() * trading_days
        # try lstm_forecast_df
        if 'lstm_forecast_df' in globals():
            fc = lstm_forecast_df['mean']
            fc_ret = fc.pct_change().dropna()
            return fc_ret.mean() * trading_days
    except Exception as e:
        print("Forecast-based TSLA return extraction failed:", e)
    return None

tsla_forecast_ann_return = tsla_expected_return_from_forecast()

# If no forecast, fallback to historical annualized mean (use recent 1 year for forward-looking bias)
if tsla_forecast_ann_return is None:
    recent_days = 252  # last 1 year
    tsla_hist = daily_returns['TSLA'].tail(recent_days)
    tsla_forecast_ann_return = tsla_hist.mean() * trading_days
    print("No forecast found — using recent historical annualized return for TSLA as proxy.")

# BND & SPY historical annualized mean returns (use full history)
bnd_ann_return = daily_returns['BND'].mean() * trading_days
spy_ann_return = daily_returns['SPY'].mean() * trading_days

expected_returns = np.array([tsla_forecast_ann_return, bnd_ann_return, spy_ann_return])
asset_names = ['TSLA', 'BND', 'SPY']

# --- 2. Covariance matrix (annualized) ---
cov_matrix_annual = daily_returns.cov() * trading_days
cov_matrix_annual = cov_matrix_annual.loc[asset_names, asset_names].values

# --- 3. Monte Carlo simulation to sample portfolios ---
def portfolio_performance(weights, exp_rets, cov_matrix):
    ret = np.dot(weights, exp_rets)
    vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return ret, vol

np.random.seed(42)
results = np.zeros((n_portfolios, 3 + len(asset_names)))  # columns: ret, vol, sharpe, weights...
for i in range(n_portfolios):
    # random weights that sum to 1 (no shorting)
    w = np.random.random(len(asset_names))
    w /= np.sum(w)
    ret, vol = portfolio_performance(w, expected_returns, cov_matrix_annual)
    sharpe = (ret - risk_free_rate) / vol if vol > 0 else 0
    results[i, 0] = ret
    results[i, 1] = vol
    results[i, 2] = sharpe
    results[i, 3:] = w

results_df = pd.DataFrame(results, columns=['Return','Volatility','Sharpe'] + asset_names)

# --- 4. Find Minimum Volatility portfolio from simulated set ---
min_vol_idx = results_df['Volatility'].idxmin()
min_vol_port = results_df.loc[min_vol_idx]

# --- 5. Find Maximum Sharpe portfolio from simulated set ---
max_sharpe_idx = results_df['Sharpe'].idxmax()
max_sharpe_port = results_df.loc[max_sharpe_idx]

# --- 6. Local optimization for more accurate Max Sharpe / Min Vol weights (constrained) ---
# Constraints & bounds
bounds = tuple((0,1) for _ in asset_names)  # no shorting
cons = ({'type':'eq', 'fun': lambda w: np.sum(w) - 1})

# Min volatility via scipy minimize
def minimize_volatility(exp_rets, cov_matrix):
    init_guess = np.array(len(asset_names)*[1/len(asset_names)])
    def fun(w):
        return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
    res = minimize(fun, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    return res.x

minvol_weights = minimize_volatility(expected_returns, cov_matrix_annual)
minvol_ret, minvol_vol = portfolio_performance(minvol_weights, expected_returns, cov_matrix_annual)
minvol_sharpe = (minvol_ret - risk_free_rate) / minvol_vol

# Max Sharpe via maximize Sharpe -> minimize negative Sharpe
def maximize_sharpe(exp_rets, cov_matrix, rf):
    init_guess = np.array(len(asset_names)*[1/len(asset_names)])
    def neg_sharpe(w):
        r, v = portfolio_performance(w, exp_rets, cov_matrix)
        return - (r - rf) / v
    res = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)
    return res.x

maxsharpe_weights = maximize_sharpe(expected_returns, cov_matrix_annual, risk_free_rate)
maxsharpe_ret, maxsharpe_vol = portfolio_performance(maxsharpe_weights, expected_returns, cov_matrix_annual)
maxsharpe_sharpe = (maxsharpe_ret - risk_free_rate) / maxsharpe_vol

# --- 7. Plot Efficient Frontier (simulated) and mark portfolios ---
plt.figure(figsize=(10,6))
plt.scatter(results_df['Volatility'], results_df['Return'], c=results_df['Sharpe'], cmap='viridis', s=8, alpha=0.6)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(minvol_vol, minvol_ret, marker='*', color='r', s=200, label='Min Vol (opt)')
plt.scatter(maxsharpe_vol, maxsharpe_ret, marker='D', color='b', s=150, label='Max Sharpe (opt)')
plt.xlabel('Annualized Volatility (Std Dev)')
plt.ylabel('Annualized Return')
plt.title('Efficient Frontier (Monte Carlo sample)')
plt.legend()
plt.grid(True)
plt.show()

# --- 8. Results summary (print) ---
def print_portfolio(name, weights, ret, vol, sharpe):
    print(f"--- {name} ---")
    for n, w in zip(asset_names, weights):
        print(f"{n}: {w:.4f}")
    print(f"Expected Annual Return: {ret:.4%}")
    print(f"Annual Volatility: {vol:.4%}")
    print(f"Sharpe Ratio: {sharpe:.4f}")
    print("")

print("From simulation (best found in simulated pool):")
print("Simulated Min Vol (approx):")
print(min_vol_port)
print("Simulated Max Sharpe (approx):")
print(max_sharpe_port)
print("\nFrom local optimization (more precise):")
print_portfolio("Minimum Volatility (optimized)", minvol_weights, minvol_ret, minvol_vol, minvol_sharpe)
print_portfolio("Maximum Sharpe (optimized)", maxsharpe_weights, maxsharpe_ret, maxsharpe_vol, maxsharpe_sharpe)

# Return important objects for further reporting if needed
optimization_results = {
    'expected_returns': expected_returns,
    'cov_matrix_annual': cov_matrix_annual,
    'simulated_portfolios': results_df,
    'minvol_optimized': {'weights': minvol_weights, 'ret': minvol_ret, 'vol': minvol_vol, 'sharpe': minvol_sharpe},
    'maxsharpe_optimized': {'weights': maxsharpe_weights, 'ret': maxsharpe_ret, 'vol': maxsharpe_vol, 'sharpe': maxsharpe_sharpe}
}
